In [1]:
!pip install flask
!pip install pyserial

In [2]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/')
def index():
    return 'Welcome to my capstone server'

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
127.0.0.1 - - [04/May/2024 19:23:02] "GET / HTTP/1.1" 200 -


In [1]:
from flask import Flask, request
import os
from werkzeug.utils import secure_filename
import subprocess

app = Flask(__name__)

# 안전한 디렉토리로 변경하세요. 예: 현재 작업 디렉토리에 uploads 폴더를 사용
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# UPLOAD_FOLDER가 존재하는지 확인하고, 없으면 생성
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        save_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(save_path)
        
        # 파일 저장 후 파이썬 스크립트 실행, 시스템에 맞게 'python' 또는 'python3'을 사용하세요.
        subprocess.run(['python3', 'test1.py', save_path])
        
        return 'File uploaded and script executed'
    return 'File type not allowed'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit
127.0.0.1 - - [04/May/2024 19:15:19] "GET / HTTP/1.1" 405 -
127.0.0.1 - - [04/May/2024 19:15:20] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.27 - - [04/May/2024 19:15:30] "GET / HTTP/1.1" 405 -
192.168.0.27 - - [04/May/2024 19:15:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/May/2024 19:15:42] "GET / HTTP/1.1" 405 -


In [5]:
from flask import Flask
import serial
import threading

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM3', 9600)

def read_from_port(ser):
    while True:
        reading = ser.readline().decode()
        print("Received from Arduino:", reading)

thread = threading.Thread(target=read_from_port, args=(ser,))
thread.start()

@app.route('/')
def index():
    return "Serial Communication with Arduino"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.27:5000
Press CTRL+C to quit


Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!



127.0.0.1 - - [06/May/2024 18:48:31] "GET / HTTP/1.1" 200 -


Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received from Arduino: Hello, Flask!

Received fro

In [10]:
from flask import Flask, request
import easyocr
import os
import subprocess
import serial

app = Flask(__name__)

# 아두이노와 통신을 위한 Serial 포트 설정
# COM 포트와 보드레이트는 프로젝트에 맞게 조정해야 합니다.
arduino = serial.Serial('COM3', 9600)

@app.route('/upload', methods=['POST'])
def save_image():
    if 'photo' in request.files:
        photo = request.files['photo']
        save_path = os.path.join('uploads', 'photo.jpg')
        photo.save(save_path)
        ocr_result = run_ocr(save_path)

        # OCR 결과를 아두이노로 보내기 전에 외부 스크립트 실행
        script_result = run_script()
        send_text_to_arduino(script_result)
        
        return ocr_result
    return 'No photo uploaded', 400

def run_ocr(image_path):
    reader = easyocr.Reader(['en','ko'])
    result = reader.readtext(image_path)
    text = ' '.join([elem[1] for elem in result])
    return text

def run_script():
    # 'main.py'는 변환된 Jupyter Notebook 스크립트의 경로입니다.
    # 실행 결과를 캡처하고 싶다면, stdout과 stderr를 subprocess.PIPE로 설정해야 합니다.
    result = subprocess.run(['python', 'C:\\Users\\peter\\capstone\\main.py'], capture_output=True, text=True)
    return result.stdout

def send_text_to_arduino(text):
    # 아두이노에 텍스트 전송
    arduino.write(text.encode())

if __name__ == '__main__':
    app.run(debug=True)


SerialException: could not open port 'COM3': PermissionError(13, '액세스가 거부되었습니다.', None, 5)